In [5]:
import keras
import sys
import math
import time
import glob
import numpy as np
from PIL import Image
import tensorflow as tf
from numpy import array
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout

In [2]:
Imgs1 = glob.glob("Data/*.jpg")

In [3]:
faces_imgs = np.load("faces_imgs.npy")
faces_annot = np.load("faces_annot.npy")[0]

In [4]:
def get_face_id(img):
    faces = []
    for i in faces_imgs:
        if i[1] == img:
            faces.append(i[0])
#     print(faces)
    return np.array(faces)

In [6]:
def get_id_coord(img):
#     print(img)
    W, H = Image.open(img).size
    coord = [] 
    ids = get_face_id(img[5:])
#     print(ids)
    for i in ids:
        if int(i) not in faces_annot.keys():
            return np.array([])
        coord.append(faces_annot[int(i)])
    a = np.array(coord, dtype=np.float32).reshape(-1, 4)
    a = a[a[:,0].argsort()]
    a[:, 0] = a[:, 0] /  W 
    a[:, 1] = a[:, 1] /  H 
    a[:, 2] = a[:, 2] /  W 
    a[:, 3] = a[:, 3] /  H 

    a = np.append(arr = np.ones((len(a), 1)), values= a, axis = 1).reshape(-1, 5)
    return a

In [ ]:
Imgs = np.load("imgsFINAL.npy")[0]

In [ ]:
imgs = glob.glob("train/*/*.jpg")

In [ ]:
Imgs['train/0--Parade/0_Parade_Parade_0_181.jpg']

In [ ]:
def coord(imgName):
    W, H = Image.open(imgName).convert("RGB").size
    a = Imgs[imgName]
    a = a[a[:,1].argsort()]
    a[:, 1] = a[:, 1] /  W 
    a[:, 2] = a[:, 2] /  H 
    a[:, 3] = a[:, 3] /  W 
    a[:, 4] = a[:, 4] /  H 
    
    return a

In [7]:
K.set_learning_phase(1)
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.85)
sess = tf.Session()

X = tf.placeholder(dtype = tf.float32, shape = (1, None, None, 3))
Y = tf.placeholder(dtype = tf.float32, shape = (None, 5))

In [8]:
vgg = VGGFace(include_top=False, input_tensor = X, input_shape=(None, None, 3))
for layer in vgg.layers:
    layer.trainable = False

In [9]:
p1 = Conv2D(filters=1024, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=11).output)
p1 = BatchNormalization()(p1)
p1 = Activation('relu')(p1)
p21 = Conv2D(filters=1024, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=13).output)
p2 = BatchNormalization()(p21)
p2 = Activation('relu')(p2)

p101 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p102 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p102 = BatchNormalization()(p102)
# p102 = Activation('relu')(p102)
# p102 = Activation('sigmoid')(p102)
p201 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p202 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p202 = BatchNormalization()(p202)
# p202 = Activation('relu')(p202)
# p202 = Activation('sigmoid')(p202)


# DownSample

p1 = MaxPool2D(strides= 2)(p1)
p2 = MaxPool2D(strides= 2)(p2)

p1 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(p1)
p1 = BatchNormalization()(p1)
p1 = Activation('relu')(p1)
p2 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(p2)
p2 = BatchNormalization()(p2)
p2 = Activation('relu')(p2)

p111 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p112 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p112 = BatchNormalization()(p112)
# p112 = Activation('relu')(p112)
# p112 = Activation('sigmoid')(p112)
p211 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p212 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p212 = BatchNormalization()(p212)
# p212 = Activation('relu')(p212)
# p212 = Activation('sigmoid')(p212)

# DownSample

p1 = MaxPool2D(strides= 2)(p1)
p2 = MaxPool2D(strides= 2)(p2)

p1 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(p1)
p1 = BatchNormalization()(p1)
p1 = Activation('relu')(p1)
p2 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(p2)
p2 = BatchNormalization()(p2)
p2 = Activation('relu')(p2)

p121 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p122 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p122 = BatchNormalization()(p122)
# p122 = Activation('relu')(p122)
# p122 = Activation('sigmoid')(p122)
p221 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p222 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p222 = BatchNormalization()(p222)
# p222 = Activation('relu')(p222)
# p222 = Activation('sigmoid')(p222)

# DownSample

p1 = MaxPool2D(strides= 2)(p1)
p2 = MaxPool2D(strides= 2)(p2)

p1 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(p1)
p1 = BatchNormalization()(p1)
p1 = Activation('relu')(p1)
p2 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(p2)
p2 = BatchNormalization()(p2)
p2 = Activation('relu')(p2)

p131 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p132 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p132 = BatchNormalization()(p132)
# p132 = Activation('relu')(p132)
# p132 = Activation('sigmoid')(p132)
p231 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p232 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p232 = BatchNormalization()(p232)
# p232 = Activation('relu')(p232)
# p232 = Activation('sigmoid')(p232)

# Combine All Predictions

p102 = K.concatenate(tensors=[p101, p102], axis = -1)
p202 = K.concatenate(tensors=[p201, p202], axis = -1)

p112 = K.concatenate(tensors=[p111, p112], axis = -1)
p212 = K.concatenate(tensors=[p211, p212], axis = -1)

p122 = K.concatenate(tensors=[p121, p122], axis = -1)
p222 = K.concatenate(tensors=[p221, p222], axis = -1)

p132 = K.concatenate(tensors=[p131, p132], axis = -1)
p232 = K.concatenate(tensors=[p231, p232], axis = -1)

In [10]:
preds = [K.reshape(p102, (-1, 5)),
        K.reshape(p202, (-1, 5)),
        K.reshape(p112, (-1, 5)),
        K.reshape(p212, (-1, 5)),
        K.reshape(p122, (-1, 5)),
        K.reshape(p222, (-1, 5)),
        K.reshape(p132, (-1, 5)),
        K.reshape(p232, (-1, 5))
       ]

In [11]:
preds = K.concatenate(preds, axis = 0)

In [12]:
def NMS(boxes = preds):    
    Z0 = tf.image.non_max_suppression(boxes=K.reshape(boxes[:, 1:], (-1, 4)), scores=K.reshape(boxes[:, 0], (-1, )),
                                        max_output_size=40, iou_threshold=0.55)
    A = K.reshape(tf.gather_nd(boxes, K.reshape(Z0, (-1, 1))), (-1, 5))
    return A

In [16]:
def loss(Y, y, a=0.8):
    if tf.size(Y) != tf.size(y):
        n = tf.truncatediv(tf.size(Y),  K.constant(value = 5, dtype=tf.int32))
        y_ = y[:n]
        
    softMax = K.mean(-(Y[:, 0] * K.log(tf.clip_by_value(y_[:, 0], 1e-10,1.0)) + (1 - Y[:, 0]) * K.log(tf.clip_by_value((1 - y_[:, 0]), 1e-10,1.0)))) + 0.85 * K.mean(-(tf.zeros_like(y[n:])[:, 0] * K.log(tf.clip_by_value(y[n:][:, 0], 1e-10,1.0)) + (1 - tf.zeros_like(y[n:])[:, 0]) * K.log(tf.clip_by_value((1 - y[n:][:, 0]), 1e-10,1.0))))
    mse = K.mean(K.square(Y[:, 1:] - y_[:, 1:]))
    return softMax + 1.35 * mse

In [17]:
bbox = NMS()

In [18]:
cost = loss(Y, bbox)

In [19]:
opt = tf.train.AdamOptimizer(learning_rate=0.0000025)
train_ = opt.minimize(cost)

In [20]:
saver = tf.train.Saver()

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
sess.run(tf.global_variables_initializer())

In [24]:
def train(ID, it, epochs = 10):
    for e in range(epochs):
        loss = 0.0
        for i in range(len(Imgs1)):            
            try:
#                 y1 = coord(imgs[i])
#                 if len(y1) == 0:
#                     continue                
#                 if len(y1) > 40:
#                     len_ERR += 1 
#                 b, L, _ = sess.run([bbox, cost, train_], feed_dict={X: np.expand_dims(np.array(Image.open(imgs[i]).convert("RGB").resize((60, 60), Image.ANTIALIAS))/255, axis = 0), Y: y1})
#                 loss += L
                y1 = get_id_coord(Imgs1[i])
                if len(y1) == 0:
                    continue
                if len(y1) > 40:
                    len_ERR += 1 
                b, L, _ = sess.run([bbox, cost, train_], feed_dict={X: np.expand_dims(np.array(Image.open(Imgs1[i]).convert("RGB").resize((60, 60), Image.ANTIALIAS))/255, axis = 0), Y: y1})
                loss += L
                if math.isnan(L):
                    print("NAN Occured")
                    return
                if i % 5000 == 0:
                    print("iter {} loss: {}".format(i, L))
                if i % 50 == 0:
                    file = open("ckpt_{}_loss.txt".format(e+1), "a")
                    file.write("iter {} loss: {}\n".format(i, L))
                    file.close()
                    
                if i % 2000 == 0:
                    saver.save(sess, save_path="checkpoints/epoch_{}_iter_{}.ckpt".format(e+1, i))
            except:
                continue

In [ ]:
train(0, 0, 10)

iter 0 loss: 0.02835824526846409


/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 0. 
  warnings.warn(str(msg))


iter 5000 loss: 0.028718966990709305


/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19660800 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))
/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


iter 10000 loss: 0.028140893206000328


...

iter 15000 loss: 0.035236768424510956


/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


iter 20000 loss: 0.029017340391874313


/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 99811328 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 70385664 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 7595. Skipping tag 7
  " Skipping tag %s" % (size, len(data), tag))
/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 131073 bytes but only got 7591. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/chawat/.local/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:725: UserWar

iter 0 loss: 0.03239038214087486
iter 5000 loss: 0.030386656522750854


In [22]:
saver.restore(sess, save_path="checkpoints/epoch_5_iter_20000.ckpt")

INFO:tensorflow:Restoring parameters from checkpoints/epoch_5_iter_20000.ckpt


In [ ]:
def plot_box(bbox, imgName):
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from PIL import Image
    %matplotlib inline
    import numpy as np
#     w, h = 500, 500
    im = np.array(Image.open(imgName), dtype=np.uint8)
    fig,ax = plt.subplots(1)
    ax.imshow(im)
    for box in bbox:
        _, X, Y, W, H = box
        rect = patches.Rectangle((X  ,Y ), W , H ,linewidth=3,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
        
    plt.savefig("image.png")

In [ ]:
def plot_box1(bbox, imgName):
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    from PIL import Image
    import numpy as np
    w, h = 100, 100
    im = np.array(Image.open(imgName).resize((100, 100), Image.ANTIALIAS), dtype=np.uint8)
    fig,ax = plt.subplots(1)
    ax.imshow(im)
    for box in bbox:
        _, X, Y, W, H = box
        rect = patches.Rectangle((X * w ,Y * h), W * w, H * h ,linewidth=3,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
        
    plt.show()

In [ ]:
imgs[0]

In [ ]:
plot_box()

In [ ]:
Imgs["train/37--Soccer/37_Soccer_Soccer_37_520.jpg"]

In [ ]:
from matplotlib import interactive
interactive(True)

In [ ]:
g = 23

In [ ]:
d = np.abs(sess.run(bbox, feed_dict={X: np.expand_dims(np.array(Image.open("face.jpg").convert("RGB").resize((100, 100), Image.ANTIALIAS))/255, axis = 0)}))

In [ ]:
d = d[d[:, 0] > 0.55]

In [ ]:
d

In [ ]:
plot_box1(d[:], "face.jpg")